In [5]:
from tracemalloc import Snapshot
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from scipy import stats
import ast 
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity

# Loading data from files

In [6]:
apps = pd.read_csv('jrs_data/apps.tsv', delimiter='\t',encoding='utf-8')
user_history = pd.read_csv('jrs_data/user_history.tsv', delimiter='\t',encoding='utf-8')
jobs = pd.read_csv('jrs_data/jobs.tsv', delimiter='\t',encoding='utf-8', error_bad_lines=False)
users = pd.read_csv('jrs_data/users.tsv' ,delimiter='\t',encoding='utf-8')
test_users = pd.read_csv('jrs_data/test_users.tsv', delimiter='\t',encoding='utf-8')

C:\Users\btr\AppData\Local\Temp\ipykernel_10568\60438194.py:3: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  jobs = pd.read_csv('jrs_data/jobs.tsv', delimiter='\t',encoding='utf-8', error_bad_lines=False)
Skipping line 122433: expected 11 fields, saw 12

Skipping line 602576: expected 11 fields, saw 12

Skipping line 990950: expected 11 fields, saw 12

C:\Users\btr\AppData\Local\Temp\ipykernel_10568\60438194.py:3: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  jobs = pd.read_csv('jrs_data/jobs.tsv', delimiter='\t',encoding='utf-8', error_bad_lines=False)


In [3]:
apps.head()

,UserID,WindowID,Split,ApplicationDate,JobID
0,47,1,Train,2012-04-04 15:56:23.537,169528
1,47,1,Train,2012-04-06 01:03:00.003,284009
2,47,1,Train,2012-04-05 02:40:27.753,2121
3,47,1,Train,2012-04-05 02:37:02.673,848187
4,47,1,Train,2012-04-05 22:44:06.653,733748


In [4]:
user_history.head()

,UserID,WindowID,Split,Sequence,JobTitle
0,47,1,Train,1,National Space Communication Programs-Special ...
1,47,1,Train,2,Detention Officer
2,47,1,Train,3,"Passenger Screener, TSA"
3,72,1,Train,1,"Lecturer, Department of Anthropology"
4,72,1,Train,2,Student Assistant


In [5]:
jobs.head()

,JobID,WindowID,Title,Description,Requirements,City,State,Country,Zip5,StartDate,EndDate
0,1,1,Security Engineer/Technical Lead,<p>Security Clearance Required:&nbsp; Top Secr...,<p>SKILL SET</p>\r<p>&nbsp;</p>\r<p>Network Se...,Washington,DC,US,20531,2012-03-07 13:17:01.643,2012-04-06 23:59:59
1,4,1,SAP Business Analyst / WM,<strong>NO Corp. to Corp resumes&nbsp;are bein...,<p><b>WHAT YOU NEED: </b></p>\r<p>Four year co...,Charlotte,NC,US,28217,2012-03-21 02:03:44.137,2012-04-20 23:59:59
2,7,1,P/T HUMAN RESOURCES ASSISTANT,<b> <b> P/T HUMAN RESOURCES ASSISTANT</b> <...,Please refer to the Job Description to view th...,Winter Park,FL,US,32792,2012-03-02 16:36:55.447,2012-04-01 23:59:59
3,8,1,Route Delivery Drivers,CITY BEVERAGES Come to work for the best in th...,Please refer to the Job Description to view th...,Orlando,FL,US,NaN,2012-03-03 09:01:10.077,2012-04-02 23:59:59
4,9,1,Housekeeping,I make sure every part of their day is magica...,Please refer to the Job Description to view th...,Orlando,FL,US,NaN,2012-03-03 09:01:11.88,2012-04-02 23:59:59


# Cleaning the text columns

In [133]:
removePattern = r'(<(.*?)>)|(&\w+)'
addSpacePattern = r'([;:])|(\\r)|(\\n)'
removeExtraSpaces = r'(\s\s+?)(?=\S)'
jobs['DescCleaned'] = jobs['Description'].str.lower()
jobs['DescCleaned'] = jobs['DescCleaned'].str.replace(removePattern, "")
jobs['DescCleaned'] = jobs['DescCleaned'].str.replace(addSpacePattern, " ")
jobs['DescCleaned'] = jobs['DescCleaned'].str.replace(removeExtraSpaces, " ")
# Same for Requirements
jobs['ReqCleaned'] = jobs['Requirements'].str.lower()
jobs['ReqCleaned'] = jobs['ReqCleaned'].str.replace(removePattern, "")
jobs['ReqCleaned'] = jobs['ReqCleaned'].str.replace(addSpacePattern, " ")
jobs['ReqCleaned'] = jobs['ReqCleaned'].str.replace(removeExtraSpaces, " ")

C:\Users\btr\AppData\Local\Temp\ipykernel_30896\2104921561.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  jobs['DescCleaned'] = jobs['DescCleaned'].str.replace(removePattern, "")
C:\Users\btr\AppData\Local\Temp\ipykernel_30896\2104921561.py:6: FutureWarning: The default value of regex will change from True to False in a future version.
  jobs['DescCleaned'] = jobs['DescCleaned'].str.replace(addSpacePattern, " ")
C:\Users\btr\AppData\Local\Temp\ipykernel_30896\2104921561.py:7: FutureWarning: The default value of regex will change from True to False in a future version.
  jobs['DescCleaned'] = jobs['DescCleaned'].str.replace(removeExtraSpaces, " ")
C:\Users\btr\AppData\Local\Temp\ipykernel_30896\2104921561.py:10: FutureWarning: The default value of regex will change from True to False in a future version.
  jobs['ReqCleaned'] = jobs['ReqCleaned'].str.replace(removePattern, "")
C:\Users\btr\AppData\Local\Temp\ipykernel_30896\210492156

In [134]:
jobs.head()

,JobID,WindowID,Title,Description,Requirements,City,State,Country,Zip5,StartDate,EndDate,DescCleaned,ReqCleaned
0,1,1,Security Engineer/Technical Lead,<p>Security Clearance Required:&nbsp; Top Secr...,<p>SKILL SET</p>\r<p>&nbsp;</p>\r<p>Network Se...,Washington,DC,US,20531,2012-03-07 13:17:01.643,2012-04-06 23:59:59,security clearance required top secret job num...,skill set network security tools webdefend web...
1,4,1,SAP Business Analyst / WM,<strong>NO Corp. to Corp resumes&nbsp;are bein...,<p><b>WHAT YOU NEED: </b></p>\r<p>Four year co...,Charlotte,NC,US,28217,2012-03-21 02:03:44.137,2012-04-20 23:59:59,no corp. to corp resumes are being considered ...,what you need four year college degree minimum...
2,7,1,P/T HUMAN RESOURCES ASSISTANT,<b> <b> P/T HUMAN RESOURCES ASSISTANT</b> <...,Please refer to the Job Description to view th...,Winter Park,FL,US,32792,2012-03-02 16:36:55.447,2012-04-01 23:59:59,p/t human resources assistant —— 1-2 years ex...,please refer to the job description to view th...
3,8,1,Route Delivery Drivers,CITY BEVERAGES Come to work for the best in th...,Please refer to the Job Description to view th...,Orlando,FL,US,NaN,2012-03-03 09:01:10.077,2012-04-02 23:59:59,city beverages come to work for the best in th...,please refer to the job description to view th...
4,9,1,Housekeeping,I make sure every part of their day is magica...,Please refer to the Job Description to view th...,Orlando,FL,US,NaN,2012-03-03 09:01:11.88,2012-04-02 23:59:59,i make sure every part of their day is magical...,please refer to the job description to view th...


In [6]:
users.head()

,UserID,WindowID,Split,City,State,Country,ZipCode,DegreeType,Major,GraduationDate,WorkHistoryCount,TotalYearsExperience,CurrentlyEmployed,ManagedOthers,ManagedHowMany
0,47,1,Train,Paramount,CA,US,90723,High School,NaN,1999-06-01 00:00:00,3,10.0,Yes,No,0
1,72,1,Train,La Mesa,CA,US,91941,Master's,Anthropology,2011-01-01 00:00:00,10,8.0,Yes,No,0
2,80,1,Train,Williamstown,NJ,US,08094,High School,Not Applicable,1985-06-01 00:00:00,5,11.0,Yes,Yes,5
3,98,1,Train,Astoria,NY,US,11105,Master's,Journalism,2007-05-01 00:00:00,3,3.0,Yes,No,0
4,123,1,Train,Baton Rouge,LA,US,70808,Bachelor's,Agricultural Business,2011-05-01 00:00:00,1,9.0,Yes,No,0


# Filtering the data using WindowID==1

In [7]:
apps_training = apps[(apps["WindowID"]==1) & (apps["Split"]=="Train")]
apps_test = apps[(apps["WindowID"]==1) & (apps["Split"]=="Test")]
user_history_training = user_history[(user_history["WindowID"]==1) & (user_history["Split"]=="Train")]
user_history_test = user_history[(user_history["WindowID"]==1) & (user_history["Split"]=="Test")]
user_training = users[(users["WindowID"]==1) & (users["Split"]=="Train")]
user_test = users[(users["WindowID"]==1) & (users["Split"]=="Test")]

In [8]:
user_ids_test = set(list(user_test["UserID"].unique()))
user_ids_train = set(list(user_training["UserID"].unique()))
apps_ids_train = set(list(apps_training["UserID"].unique()))
apps_ids_test = set(list(apps_test["UserID"].unique()))
print(len(user_ids_test))
print(len(apps_ids_test))
print(len(apps_ids_train))
print(len(user_ids_test-apps_ids_test))
print(len(apps_ids_test - apps_ids_train))
print(len(user_ids_test-user_ids_train))


5419
3138
60274
2281
3138
5419


In [9]:
print(user_training["DegreeType"].nunique())
print(user_training["Major"].nunique())
print(user_training["Country"].nunique())
user_training = user_training.iloc[0:20000,:]
user_training["DegreeType"] = user_training["DegreeType"].fillna("")
user_training["Major"] = user_training["Major"].fillna("")
user_training["TotalYearsExperience"] = user_training["TotalYearsExperience"].fillna(0)

7
12027
67


In [10]:
user_training.shape

(20000, 15)

# Convert "Major" column into TF-IDF vectors

In [11]:
tf = TfidfVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(user_training['Major'])
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=tf.get_feature_names())

c:\Users\btr\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [12]:
print(len(list(tf.get_feature_names())))

5684


c:\Users\btr\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


# Concat numerical feature, categorical features and tf-idf vectors for train data

In [13]:
cat_feats = ["DegreeType","CurrentlyEmployed","ManagedOthers"]
num_feats = ["WorkHistoryCount","TotalYearsExperience","ManagedHowMany"]
for col in cat_feats:
    user_training[col] = user_training[col].astype("category")
user_training_cat = user_training[cat_feats]
user_training_num = user_training[num_feats]
user_training_cat = pd.get_dummies(user_training_cat,drop_first=True)
user_training_num_cat = pd.concat([tfidf_df.reset_index(drop=True),user_training_num.reset_index(drop=True),user_training_cat.reset_index(drop=True)],axis=1,ignore_index=True)


# Scaling different features onto same scale

In [14]:
from sklearn.preprocessing import MinMaxScaler
def scaler(X):
    scale = MinMaxScaler()
    colnames = X.select_dtypes(include=['float64','int64']).columns.to_list()
    #colnames.remove("salary") #salary is target variable
    print('\nThe following columns are scaled:\n')
    #print(colnames)
    scale.fit(X[colnames])
    X[colnames] =  scale.transform(X[colnames])
    return X,scale

In [15]:
user_training_num_cat.info()
user_training_num_cat,scaler_user = scaler(user_training_num_cat)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Columns: 5695 entries, 0 to 5694
dtypes: float64(5685), int64(2), uint8(8)
memory usage: 867.9 MB

The following columns are scaled:



In [16]:
user_training_num_cat.isnull().sum()

0       0
1       0
2       0
3       0
4       0
       ..
5690    0
5691    0
5692    0
5693    0
5694    0
Length: 5695, dtype: int64

# Use K nearest neighbour for similar user search

In [17]:
from sklearn.neighbors import NearestNeighbors
neigh_users = NearestNeighbors(n_neighbors=5, n_jobs=-1) 
neigh_users.fit(user_training_num_cat)

NearestNeighbors(n_jobs=-1)

In [18]:
user_test.shape

(5419, 15)

# Processing for test users

In [25]:
user_test["DegreeType"] = user_test["DegreeType"].fillna("")
user_test["Major"] = user_test["Major"].fillna("")
user_test["TotalYearsExperience"] = user_test["TotalYearsExperience"].fillna(0)

# Tranform test users using TF-IDF(use "Major" column)

In [20]:
tfidf_matrix = tf.transform(user_test['Major'])
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=tf.get_feature_names())

c:\Users\btr\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


# Concat numerical feature, categorical features and tf-idf vectors for test data

In [21]:
for col in cat_feats:
    user_test[col] = user_test[col].astype("category")
user_test_cat = user_test[cat_feats]
user_test_num = user_test[num_feats]
user_test_cat = pd.get_dummies(user_test_cat,drop_first=True)
user_test_cat = user_test_cat.reindex(columns = user_training_cat.columns,fill_value=0)
user_test_num_cat = pd.concat([tfidf_df.reset_index(drop=True),user_test_num.reset_index(drop=True),user_test_cat.reset_index(drop=True)],axis=1,ignore_index=True)

C:\Users\btr\AppData\Local\Temp\ipykernel_30896\4083328718.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  user_test[col] = user_test[col].astype("category")
C:\Users\btr\AppData\Local\Temp\ipykernel_30896\4083328718.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  user_test[col] = user_test[col].astype("category")
C:\Users\btr\AppData\Local\Temp\ipykernel_30896\4083328718.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_

# Tranform test user using training scaler

In [22]:
print(user_training_num_cat.shape)
print(user_test_num_cat.shape)
colnames = user_test_num_cat.select_dtypes(include=['float64','int64']).columns.to_list()
user_test_num_cat[colnames] = scaler_user.transform(user_test_num_cat[colnames])


(20000, 5695)
(5419, 5695)


# Function to perform collaborative filtering using similar users

In [28]:
def get_cfr(users_input):
    res = neigh_users.kneighbors(users_input, return_distance=False)
    ind = 0
    users_recommendation={}
    for sim_users in res:
        print(user_test.iloc[ind:ind+1,:])
        sim_user_ids = list()
        for sim_user in sim_users:
            sim_user_ids.append(int(user_training.iloc[sim_user:sim_user+1,:]["UserID"]))
            print(user_training.iloc[sim_user:sim_user+1,:])
        print(sim_user_ids)
        df1 = apps_training[apps_training['UserID'].isin(sim_user_ids)][['JobID']]
        joblist = df1['JobID'].tolist()
        #print("listed jobs are: ")
        #print(joblist)
        df_temp = jobs[jobs['JobID'].isin(joblist)][['JobID','Title','Description','City','State']]
        #print(user_test.iloc[ind:ind+1,:]["UserID"])
        #users_recommendation[int(user_test.iloc[ind:ind+1,:]["UserID"])] = joblist
        print(df_temp)
        #input()
        if ind>=4:
            break
        ind+=1
    return users_recommendation

user_recommendation = get_cfr(user_test_num_cat)

    UserID  WindowID Split      City State Country ZipCode  DegreeType  \
31     767         1  Test  Murrieta    CA      US   92562  Bachelor's   

                          Major       GraduationDate  WorkHistoryCount  \
31  University Studies/Business  2008-05-01 00:00:00                 5   

    TotalYearsExperience CurrentlyEmployed ManagedOthers  ManagedHowMany  
31                  16.0                No            No               0  
      UserID  WindowID  Split     City State Country ZipCode  DegreeType  \
3770   87637         1  Train  Watauga    TX      US   76148  Bachelor's   

                   Major       GraduationDate  WorkHistoryCount  \
3770  University Studies  2011-01-01 00:00:00                 2   

      TotalYearsExperience CurrentlyEmployed ManagedOthers  ManagedHowMany  
3770                  14.0                No            No               0  
       UserID  WindowID  Split       City State Country ZipCode  DegreeType  \
18762  373207         1  Train 

# Filter jobs using WindowID==1

In [10]:
jobs = pd.read_csv('jrs_data/jobs.tsv', delimiter='\t',encoding='utf-8', error_bad_lines=False)
jobs_US= jobs[(jobs["WindowID"]==1) & (jobs["Country"]=="US")]
jobs_US = jobs_US.iloc[0:10000,0:8]

C:\Users\btr\AppData\Local\Temp\ipykernel_10568\1408448533.py:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  jobs = pd.read_csv('jrs_data/jobs.tsv', delimiter='\t',encoding='utf-8', error_bad_lines=False)
Skipping line 122433: expected 11 fields, saw 12

Skipping line 602576: expected 11 fields, saw 12

Skipping line 990950: expected 11 fields, saw 12

C:\Users\btr\AppData\Local\Temp\ipykernel_10568\1408448533.py:1: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  jobs = pd.read_csv('jrs_data/jobs.tsv', delimiter='\t',encoding='utf-8', error_bad_lines=False)


In [11]:
jobs_US.head()

,JobID,WindowID,Title,Description,Requirements,City,State,Country
0,1,1,Security Engineer/Technical Lead,<p>Security Clearance Required:&nbsp; Top Secr...,<p>SKILL SET</p>\r<p>&nbsp;</p>\r<p>Network Se...,Washington,DC,US
1,4,1,SAP Business Analyst / WM,<strong>NO Corp. to Corp resumes&nbsp;are bein...,<p><b>WHAT YOU NEED: </b></p>\r<p>Four year co...,Charlotte,NC,US
2,7,1,P/T HUMAN RESOURCES ASSISTANT,<b> <b> P/T HUMAN RESOURCES ASSISTANT</b> <...,Please refer to the Job Description to view th...,Winter Park,FL,US
3,8,1,Route Delivery Drivers,CITY BEVERAGES Come to work for the best in th...,Please refer to the Job Description to view th...,Orlando,FL,US
4,9,1,Housekeeping,I make sure every part of their day is magica...,Please refer to the Job Description to view th...,Orlando,FL,US


# clean Title and Description, and append both into Description column

In [35]:
jobs_US["Title"] = jobs_US["Title"].fillna("")
jobs_US["Description"] = jobs_US["Description"].fillna("")
#jobs_US["Requirements"] = jobs_US["Requirements"].fillna("")
jobs_US["Description"] = jobs_US["Title"] + " "+ jobs_US["Description"] #+ " "+ jobs_US["Requirements"]


# Fit and Generate TF-IDF vectors for the jobs(using Description)

In [36]:
tf = TfidfVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
tfidf_matrix_jobs = tf.fit_transform(jobs_US['Description'])
#tfidf_matrix_jobs_df = pd.DataFrame(tfidf_matrix_jobs.toarray(), columns=tf.get_feature_names())

# Nearest Neighbor search for the similar jobs

In [37]:
from sklearn.neighbors import NearestNeighbors
neigh_users = NearestNeighbors(n_neighbors=5, n_jobs=-1) 
neigh_users.fit(tfidf_matrix_jobs)

NearestNeighbors(n_jobs=-1)

In [38]:
user_test = user_test[user_test["Country"]=="US"]
user_test.head()
user_test.shape

(5404, 15)

# Function to perform content based recommendation using similar job profiles

In [55]:
def get_cbr(user_test):
    ind = 0
    for _,user_row in user_test.iterrows():
        print(user_row["UserID"])
        user_applied_job_ids = list(apps_test[apps_test["UserID"]==int(user_row["UserID"])]["JobID"].unique())[:10]
        user_profile = user_row["DegreeType"] + " "+ user_row["Major"]
        user_jobs_text = ""
        user_jobs_text = " ".join(list(jobs_US[jobs_US["JobID"].isin(user_applied_job_ids)]["Description"].dropna().unique()))
        #print(user_jobs_text)
        user_history_text = " ".join(list(user_history_test[user_history_test["UserID"]==int(user_row["UserID"])]["JobTitle"].dropna().unique()))
        user_profile += (" "+ user_history_text)
        print(user_profile)
        #print(user_history_text)
        tf_idf_user = tf.transform([user_profile])
        job_recommend_inds = neigh_users.kneighbors(tf_idf_user, return_distance=False)
        print(list(job_recommend_inds[0]))
        for id in list(job_recommend_inds[0]):
            print(jobs_US.iloc[id:id+1,:]["Title"])
        if ind>=5:
            break
        ind+=1

In [56]:
get_cbr(user_test)

767
Bachelor's University Studies/Business Claims Adjuster Professional Baseball Player
[3814, 2682, 8882, 7177, 5064]
3817    c.n.a.
Name: Title, dtype: object
2684    Sr Claims Adjuster-Minneapolis
Name: Title, dtype: object
8885    Claims Coordinator / Adjuster Trainee
Name: Title, dtype: object
7180    IT Professional
Name: Title, dtype: object
5067    A/P Professional
Name: Title, dtype: object
769
Bachelor's Radio-Television Shoe Fit Specialist WGCI, Programming Intern Hip Hop Director/DJ Teller Head Cashier
[3814, 9451, 2370, 2371, 2361]
3817    c.n.a.
Name: Title, dtype: object
9454    Programming Specialist
Name: Title, dtype: object
2372    Teller
Name: Title, dtype: object
2373    Teller
Name: Title, dtype: object
2363    Teller
Name: Title, dtype: object
861
High School General Studies Building Manager Independent Avon Rep Assistant Manager Crew Manage Office Manager Bartender
[3814, 3543, 5932, 4422, 9560]
3817    c.n.a.
Name: Title, dtype: object
3546    Office Manager
Na